This is a practice file, working with EEG data from the muse and manipulating it. 

In [ ]:
#PRACTICE IMPORTING DATA

#pandas is a library that makes working with large csv files faster and easier
import pandas as pd

#it is helpful to make a dictionary, to have convienient access to each data set

raw_data={
    Jacob_Practice_1:pd.read_csv("Data/Jacob/Jacob_Practice_1.csv")
    
}

#good idea to print the head of each dataset to verify that it has worked

for key in raw_data:
    print(f"\n{key}") 
    print(raw.data[key].head())